In [3]:
import pandas as pd

train_data = pd.read_csv("finall_data.csv")

labels = train_data['object'].to_list()
num_classes = len(set(labels))

texts = (train_data['Reviews']).to_list()
print(texts[0])

Лучший курс для подготовки к ЕГЭ по математике с отличным учителем ever Учёба с Тимуром = СОТОЧКА баллов за экзамен + отлично проведённое время


In [4]:
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6448 entries, 0 to 6447
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   6448 non-null   int64 
 1   Reviews      6448 non-null   object
 2   is_relevant  6448 non-null   int64 
 3   is_positive  6448 non-null   int64 
 4   object       6448 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 252.0+ KB
None


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6448 entries, 0 to 6447
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   6448 non-null   int64 
 1   Reviews      6448 non-null   object
 2   is_relevant  6448 non-null   int64 
 3   is_positive  6448 non-null   int64 
 4   object       6448 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 252.0+ KB


In [6]:
import torch
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Загрузка данных
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Функция для тренировки модели
def train_model(model, train_dataloader, val_dataloader, optimizer, epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_dataloader, total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{epochs}', unit='batch'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_dataloader)

        print(f'Training Loss: {avg_train_loss}')

        # Валидация
        model.eval()
        val_preds = []
        val_labels = []
        for batch in tqdm(val_dataloader, total=len(val_dataloader), desc=f'Validating Epoch {epoch + 1}/{epochs}', unit='batch'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].numpy()

            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
                val_preds.extend(preds)
                val_labels.extend(labels)

        val_f1 = f1_score(val_labels, val_preds, average='macro')
        print(f'Validation F1-Score: {val_f1}')

# Загрузка предобученного RuBERT
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=num_classes)

# Параметры
max_len = 128
batch_size = 32
epochs = 15
learning_rate = 2e-5

# Подготовка данных
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_len)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_len)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Инициализация оптимизатора
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Обучение модели
train_model(model, train_dataloader, val_dataloader, optimizer, epochs)


/Users/ivan/Desktop/Projects/ClearGeekBrains/random.randname-/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ivan/Desktop/Projects/ClearGeekBrains/random.randname-/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/ivan/Desktop/Projects/ClearGeekBrains/random.randname-/.venv/lib/python3.9/si

Training Loss: 0.6874271995491452


Validating Epoch 1/15: 100%|██████████| 41/41 [00:35<00:00,  1.17batch/s]


Validation F1-Score: 0.548511238174926


Epoch 2/15: 100%|██████████| 162/162 [10:46<00:00,  3.99s/batch]


...

Validation F1-Score: 0.7091196037710015


Epoch 15/15: 100%|██████████| 162/162 [09:41<00:00,  3.59s/batch]


Training Loss: 0.02472659364884524


Validating Epoch 15/15: 100%|██████████| 41/41 [00:35<00:00,  1.17batch/s]

Validation F1-Score: 0.7234548039244358


In [7]:
test_data = pd.read_csv("train_data.csv")
test_labels = test_data['object']
test_texts = (test_data['question_2'] + test_data['question_3'] + test_data['question_4'] + test_data['question_5']).to_list()

In [8]:
test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_len)

test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Валидация
model.eval()
val_preds = []
val_labels = []
for batch in tqdm(test_dataloader, total=len(test_dataloader)):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].numpy()

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
        val_preds.extend(preds)
        val_labels.extend(labels)

val_f1 = f1_score(val_labels, val_preds, average='micro')
print(f'Test F1-Score: {val_f1}')

100%|██████████| 4/4 [00:02<00:00,  1.35it/s]

Test F1-Score: 0.9611650485436893


In [9]:
model_path = "models/object/RuBert+Comments.bin"

# Сохранение дообученной модели в bin файл
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('models/object/RuBert+Comments.bin/tokenizer_config.json',
 'models/object/RuBert+Comments.bin/special_tokens_map.json',
 'models/object/RuBert+Comments.bin/vocab.txt',
 'models/object/RuBert+Comments.bin/added_tokens.json')